## **Skeletonization**

The following code computes the voxel-centerline (skeletonization) per case and segment label segmentation.

A global skeleton is computed to preserve continuity, and then intersected voxel-wise with each label mask.

This ensures anatomically labeled centerlines without losing structural connectivity.

### **Imports and Installs**

In [ ]:
!pip install scikit-image==0.18.3

  Using cached scikit-image-0.18.3.tar.gz (29.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 33.4 MB/s eta 0:00:00
  Created wheel for scikit-image: filename=scikit_image-0.18.3-cp311-cp311-linux_x86_64.whl size=37878510 sha256=964ba5b1dcc2465bd55383a78ddc96f32ddc95c983780aa5ada233fad31c8961
  Stored in directory: /root/.cache/pip/wheels/e3/f1/e0/18ca2948c8e8670f1f357bb1704d22638ea3cb247f77fe0ce2
Successfully built scikit-image
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.25.2
    Uninstalling scikit-image-0.25.2:
      Successfully uninstalled scikit-image-0.25.2


In [ ]:
import os
import nibabel as nib
import numpy as np
from skimage.morphology import skeletonize_3d

### **Google Drive connection**

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Upload files**

In [ ]:
# Define input and output directories
input_multilabel_dir = '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/Skeletonization/GT_multilabel'
input_binary_dir = '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/Skeletonization/GT'
output_base_dir = '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/Skeletonization/Processed'

### **Functions**

In [ ]:
def process_multilabel_case(multilabel_path, output_case_dir):
    """
    Process a single multilabel case by:
    1. Creating the skeletonization of the entire multilabel.
    2. Extracting and saving binary masks for each label (1-19).
    3. Calculating the skeletons for each label.

    Parameters:
        multilabel_path (str): Path to the input multilabel .nii.gz file.
        output_case_dir (str): Path to the output directory for this case.
    """
    # Load the multilabel image
    multilabel_img = nib.load(multilabel_path)
    multilabel_data = multilabel_img.get_fdata()

    # Ensure the data is in binary format for skeletonization
    binary_mask = (multilabel_data > 0).astype(np.uint8)

    # Perform 3D skeletonization for the entire mask
    skeleton = skeletonize_3d(binary_mask)

    # Save the skeleton for the entire multilabel
    skeleton_img = nib.Nifti1Image(skeleton.astype(np.uint8), multilabel_img.affine, multilabel_img.header)
    skeleton_path = os.path.join(output_case_dir, 'skeleton.nii.gz')
    nib.save(skeleton_img, skeleton_path)
    print(f"Saved skeleton for the entire multilabel at: {skeleton_path}")

    # Process and save skeletons for each label (1-19)
    for label in range(1, 20):
        label_mask = (multilabel_data == label).astype(np.uint8)
        label_skeleton = skeletonize_3d(label_mask)

        # Save the skeleton for the current label
        label_skeleton_img = nib.Nifti1Image(label_skeleton.astype(np.uint8), multilabel_img.affine, multilabel_img.header)
        label_skeleton_path = os.path.join(output_case_dir, f'label_{label}_skeleton.nii.gz')
        nib.save(label_skeleton_img, label_skeleton_path)
        print(f"Saved skeleton for label {label} at: {label_skeleton_path}")

def process_binary_case(binary_path, output_case_dir):
    """
    Process a single binary case by creating its 3D skeletonization.

    Parameters:
        binary_path (str): Path to the input binary .nii.gz file.
        output_case_dir (str): Path to the output directory for this case.
    """
    # Load the binary image
    binary_img = nib.load(binary_path)
    binary_data = binary_img.get_fdata()

    # Ensure the mask is binary
    binary_mask = (binary_data > 0).astype(np.uint8)

    # Perform 3D skeletonization
    skeleton = skeletonize_3d(binary_mask)

    # Save the skeletonized binary image
    skeleton_img = nib.Nifti1Image(skeleton.astype(np.uint8), binary_img.affine, binary_img.header)
    skeleton_path = os.path.join(output_case_dir, 'binary_skeleton.nii.gz')
    nib.save(skeleton_img, skeleton_path)
    print(f"Saved skeleton for binary mask at: {skeleton_path}")

def process_label_intersections(skeleton_path, multilabel_path, output_case_dir):
    """
    Computes binary intersections between the skeleton and each label (1-19)
    in the multilabel image and saves results for each label.

    Parameters:
        skeleton_path (str): Path to the skeleton .nii.gz file.
        multilabel_path (str): Path to the multilabel .nii.gz file.
        output_case_dir (str): Path to the output directory for this case.
    """
    # Load skeleton and multilabel images
    skeleton_img = nib.load(skeleton_path)
    multilabel_img = nib.load(multilabel_path)

    skeleton_data = skeleton_img.get_fdata()  # Skeleton is binary (0/1)
    multilabel_data = multilabel_img.get_fdata()  # Labels range from 0 to 19

    # Ensure skeleton is binary
    skeleton_binary = (skeleton_data == 1).astype(np.uint8)

    # Process and save intersections for each label (1-19)
    for label in range(1, 20):
        label_mask = (multilabel_data == label).astype(np.uint8)
        skeleton_label = (skeleton_binary & label_mask).astype(np.uint8)

        # Save the result
        label_intersection_img = nib.Nifti1Image(skeleton_label, skeleton_img.affine, skeleton_img.header)
        label_intersection_path = os.path.join(output_case_dir, f'label_{label}_intersection.nii.gz')
        nib.save(label_intersection_img, label_intersection_path)
        print(f"Saved intersection for label {label} at: {label_intersection_path}")


### **Execution**

In [ ]:
# Create the output base directory if it doesn't exist
os.makedirs(output_base_dir, exist_ok=True)

# Iterate through all files in the input directories
for file_name in os.listdir(input_multilabel_dir):
    if file_name.endswith('.nii.gz'):
        multilabel_path = os.path.join(input_multilabel_dir, file_name)

        # Create a directory for the current case
        case_name = os.path.splitext(os.path.splitext(file_name)[0])[0]  # Remove .nii.gz
        output_case_dir = os.path.join(output_base_dir, case_name)
        os.makedirs(output_case_dir, exist_ok=True)

        print(f"Processing multilabel case: {case_name}")
        process_multilabel_case(multilabel_path, output_case_dir)

        # Check if a corresponding binary file exists
        binary_path = os.path.join(input_binary_dir, file_name)
        if os.path.exists(binary_path):
            print(f"Processing binary case: {case_name}")
            process_binary_case(binary_path, output_case_dir)

            # Process intersections if both skeleton and multilabel exist
            skeleton_path = os.path.join(output_case_dir, 'binary_skeleton.nii.gz')
            process_label_intersections(skeleton_path, multilabel_path, output_case_dir)

print("Processing completed!")


Processing multilabel case: Diseased_7
Saved skeleton for the entire multilabel at: /content/drive/Shared drives/TFG Maren/Data/Skeletonization/Processed/Diseased_7/skeleton.nii.gz
Saved skeleton for label 1 at: /content/drive/Shared drives/TFG Maren/Data/Skeletonization/Processed/Diseased_7/label_1_skeleton.nii.gz
Saved skeleton for label 2 at: /content/drive/Shared drives/TFG Maren/Data/Skeletonization/Processed/Diseased_7/label_2_skeleton.nii.gz
Saved skeleton for label 3 at: /content/drive/Shared drives/TFG Maren/Data/Skeletonization/Processed/Diseased_7/label_3_skeleton.nii.gz
Saved skeleton for label 4 at: /content/drive/Shared drives/TFG Maren/Data/Skeletonization/Processed/Diseased_7/label_4_skeleton.nii.gz
Saved skeleton for label 5 at: /content/drive/Shared drives/TFG Maren/Data/Skeletonization/Processed/Diseased_7/label_5_skeleton.nii.gz
Saved skeleton for label 6 at: /content/drive/Shared drives/TFG Maren/Data/Skeletonization/Processed/Diseased_7/label_6_skeleton.nii.gz
Sav